In [505]:
import glob

genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = 28  # Which month and days result you want to visualize in table
month = 10
repetitionValue = "15x01sign4labels"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionValue}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.2

In [506]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]

                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])

                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)


(1, 189)
(10, 189)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.234375,0.251563,0.232813,0.217188,0.20625,0.21875,0.240625,0.214062,0.239063,0.25,0.248438,0.246875,0.242188,0.240625,0.210938,0.253125,0.289063,0.2375,0.245313,0.298437,0.226563,0.229688,0.217187,0.226563,0.232813,0.234375,0.23125,0.248438,0.264062,0.264063,0.234375,0.21875,0.223438,0.260938,0.28125,0.257812,0.279688,0.276562,0.245313,None,None,0.21875,0.229688,0.26875,0.221875,0.239063,0.24375,0.220313,0.2125,0.24375,0.234375,0.220312,0.265625,0.2125,0.259375,0.292188,0.259375,0.240625,None,0.225,0.235938,0.284375,0.253125,0.267188,0.246875,0.275,0.251563,0.234375,0.264062,0.271875,0.264063,0.246875,0.228125,0.26875,0.228125,0.265625,0.265625,0.26875,0.239063,0.257813,0.24375,0.225,0.285938,0.235937,0.220312,0.229688,0.217187,0.235938,0.253125,0.25,0.229688,0.203125,0.228125,0.25625,0.245313,0.254688,0.253125,0.251563,0.223438,0.264063,0.273438,0.265625,0.232813,0.220312,0.228125,0.259375,0.264063,0.264062,0.278125,0.282813,0.279688,0.248438,0.246875,0.259375,0.246875,0.26875,0.264063,0.264063,0.307812,0.246875,0.260938,0.270313,0.257813,0.279688,0.275,0.25625,0.265625,0.289063,0.260937,0.21875,0.275,0.239063,None,None,None,0.234375,0.253125,0.246875,0.232813,0.264063,0.2625,0.229688,0.2625,0.226563,0.245312,0.267188,0.246875,0.226563,0.301563,0.235938,0.21875,0.257813,0.242188,0.220312,0.24375,0.220312,0.2125,0.279688,0.242188,0.259375,0.265625,0.24375,0.228125,0.292187,0.2375,0.253125,0.271875,0.245312,0.235938,0.265625,0.2625,0.267188,0.271875,0.259375,0.276563,0.275,0.254688,0.271875,0.26875,0.248438,0.228125,0.275,0.25625,0.2375,0.271875,0.234375,0.223438,0.289063,0.242188
2,None,None,None,0.279948,0.231771,0.221354,0.222656,0.220052,0.203125,0.264323,0.223958,0.22526,0.28125,0.227865,None,0.251302,0.236979,0.240885,0.295573,0.270833,0.238281,0.207031,None,0.207031,None,0.205729,None,0.204427,0.226563,0.214844,None,None,None,0.226562,0.208333,0.229167,0.223958,0.221354,None,0.223958,0.221354,None,0.252604,0.223958,0.214844,0.277344,0.203125,0.229167,0.266927,0.220052,0.235677,0.251302,0.244792,0.210938,0.230469,0.231771,0.231771,0.222656,0.217448,None,0.240885,0.207031,0.240885,0.238281,0.269531,0.213542,0.263021,0.236979,0.220052,0.240885,0.226562,0.240885,0.268229,0.259115,0.252604,0.25,0.221354,0.227865,0.255208,0.24349,0.217448,0.268229,0.25,0.229167,None,0.205729,None,0.236979,0.21875,0.209635,0.207031,0.205729,None,0.238281,0.223958,0.226563,0.217

In [507]:
print("Top ten highest accuracies")
topTen.reverse()
for top in topTen:
    print(
        f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
    )


Top ten highest accuracies
Accuracy 0.31 : subject 1, feature:dataCV-dataFFTCV--fftData, kernelsigmoid, C:2.5
Accuracy 0.3 : subject 1, feature:dataCV-dataFFTCV--welchData, kernelsigmoid, C:2.5
Accuracy 0.3 : subject 1, feature:-dataCV, kernelsigmoid, C:2.5
Accuracy 0.3 : subject 2, feature:-dataCV, kernellinear, C:0.5
Accuracy 0.3 : subject 3, feature:dataHRCV--dataFFTCV, kernellinear, C:0.5
Accuracy 0.29 : subject 3, feature:dataHRCV-dataWCV--dataFFTCV, kernellinear, C:0.5
Accuracy 0.29 : subject 1, feature:dataFFTCV--dataHR, kernelsigmoid, C:2.5
Accuracy 0.29 : subject 1, feature:dataHRCV-dataFFTCV--dataHR, kernelsigmoid, C:2.5
Accuracy 0.29 : subject 3, feature:-dataWCV, kernellinear, C:0.5
Accuracy 0.29 : subject 6, feature:dataFFTCV--welchData, kernelrbf, C:2.5


In [508]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.24938525 0.23082741 0.23595882 0.22885101 0.23102793
 0.22867303 0.23621032 0.22430657 0.23393369]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,0.227332,0.227798,0.221202,0.246842,0.241007,0.227126,0.248142,0.229987,0.235448,0.240371,0.239493,0.24401,0.249497,0.241769,0.232313,0.236687,0.244257,0.232146,0.244322,0.264414,0.225538,0.222489,0.225582,0.223186,0.237205,0.226007,0.216862,0.219262,0.233909,0.232908,0.227332,0.219878,0.221951,0.237182,0.240871,0.232152,0.242445,0.233251,0.225268,0.236154,0.229597,0.218456,0.23609,0.235283,0.231601,0.242679,0.237716,0.239131,0.240964,0.23549,0.231736,0.232991,0.254648,0.237411,0.248245,0.240646,0.237379,0.247988,0.22148,0.238309,0.233866,0.231339,0.246353,0.227812,0.230052,0.233741,0.243483,0.220516,0.236631,0.247799,0.231967,0.233126,0.239745,0.240071,0.233782,0.235353,0.235094,0.230455,0.234222,0.237308,0.234993,0.234935,0.251483,0.225756,0.217938,0.220223,0.217025,0.23203,0.222835,0.222749,0.222335,0.223715,0.232225,0.230085,0.233391,0.224955,0.227586,0.238095,0.224545,0.233089,0.228139,0.226288,0.237205,0.221282,0.217347,0.244075,0.233775,0.232283,0.238606,0.235702,0.233021,0.219262,0.225035,0.233386,0.241042,0.228694,0.227759,0.24294,0.233286,0.229072,0.237182,0.240496,0.23151,0.242445,0.236508,0.223111,0.237145,0.234566,0.228827,0.224331,0.232438,0.224302,0.235633,0.220078,0.22839,0.230612,0.226929,0.224865,0.232572,0.227916,0.228315,0.236227,0.230853,0.220964,0.238838,0.227645,0.230794,0.239672,0.238632,0.235851,0.240372,0.232099,0.232045,0.233772,0.225463,0.231571,0.237704,0.239981,0.228483,0.247909,0.241079,0.232056,0.239061,0.232359,0.238712,0.232324,0.231063,0.228747,0.234039,0.231486,0.247055,0.227658,0.230142,0.235487,0.240125,0.239724,0.231859,0.247571,0.239462,0.231452,0.236707,0.243349,0.2325,0.246828,0.234219,0.226488,0.234194,0.23888,0.230766


Max average accuracy 0.26441428170594833
19
9.0
-dataCV
sigmoid
C: 2.5


In [509]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,1.363993,1.366788,0.884809,2.221578,1.928057,1.817011,2.23328,2.069886,1.883584,2.163343,1.676452,1.952079,2.245472,1.934149,0.929253,2.13018,1.954058,2.089313,2.198894,2.379729,1.804301,2.002405,1.35349,1.339118,1.423229,1.582049,0.867448,1.754099,1.637366,1.630358,1.363993,0.879514,1.109757,1.42309,1.445226,1.857216,1.45467,1.866007,1.35161,1.889233,1.836776,1.529189,2.124812,1.88226,1.852808,2.184107,1.66401,1.434785,2.168676,1.883919,1.622151,2.096916,1.782536,1.424463,1.98596,1.684519,1.899033,2.231891,1.55036,1.66816,1.870926,1.850712,2.217181,2.050308,2.07047,2.10367,2.191346,1.764126,1.419784,1.982394,1.623767,2.098134,2.157704,2.160638,1.870256,2.118174,2.115843,1.843638,2.107997,2.135775,1.644949,2.114413,2.263349,1.806048,1.743504,1.982003,0.651076,1.624211,1.782677,1.336493,2.001016,1.118576,1.161127,1.84068,1.400347,1.79964,1.820691,1.666667,1.122724,1.864713,1.825109,1.584017,1.423229,1.327691,1.30408,1.708524,1.636424,1.858262,1.670243,1.649913,1.398125,1.754099,1.125174,1.400316,1.205208,1.143472,1.822074,1.457639,1.633003,1.832576,1.42309,1.683472,1.85208,1.45467,1.655556,1.561774,1.660017,2.111095,1.830617,2.018977,1.627066,1.570114,1.885066,1.320469,1.370342,2.075505,2.042364,2.023788,2.093149,2.051244,1.826518,2.126045,1.615972,1.767708,1.910701,1.821157,1.846349,1.917377,1.909059,1.650955,1.92298,1.856795,1.624317,2.10395,1.803701,1.389426,1.901634,2.159833,1.827866,1.983269,1.446476,1.856446,1.912487,1.858872,1.909699,1.626267,1.848502,2.058723,1.872315,1.620399,1.976441,2.048919,1.610996,1.883892,1.921002,1.917795,2.086735,1.980571,1.676233,2.083071,2.13036,1.946788,1.859998,1.974621,2.107975,2.038393,2.107748,2.149921,1.615365


19
9.0
Max total significant accuracy then averaged 0.26441428170594833 
-dataCV
sigmoid
C: 2.5 


In [510]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
